# Fast-Trips Tutorial 3: Capacity Constraints

In [ ]:
import os,datetime
import pandas as pd

# Specify Input Networks and Demand
To start with, we will use the same networks as previous scenarios, but we will be using an expanded demand

In [ ]:
BASE_DIR         = r"/Users/elizabeth/Documents/urbanlabs/MTC/SHRP2/tutorial"

INPUT_NETWORKS   = os.path.join(BASE_DIR, r"tta/input/network-simple")
INPUT_DEMAND     = os.path.join(BASE_DIR, r"tta/input/demand-bigger")

RUN_CONFIG       = os.path.join(BASE_DIR, r"tta/input/demand-bigger/config_ft.txt")
INPUT_WEIGHTS    = os.path.join(BASE_DIR, r"tta/input/demand-bigger/pathweight_ft.txt")

# Run Fast-Trips 
This example runs fast-trips for a much larger amount of demand.  It looks like the game is over and all the basketball players want to go to the park!

Note that each bus only has a capacity of 10 and more than 10 riders will want to ride any given bus at once.  Therefore, we will run several iterations so that they will have a chance to find an alternate route.

Also note that this run will take a few minutes to complete since it is running more people and multiple iterations.


In [ ]:
demand_df = pd.read_csv(os.path.join(INPUT_DEMAND,"trip_list.txt"))
demand_df

In [ ]:
from fasttrips import Run

In [ ]:
OUTPUT_FOLDER    = r"capacity-demand"
OUTPUT_DIR       = os.path.join(BASE_DIR, "output")
ITERATIONS       = 3
PATHFINDING_TYPE = "stochastic"
OVERLAP          = "count"
CAPACITY         = True
DISPERSION       = 0.5

In [ ]:
Run.run_fasttrips(input_network_dir    = INPUT_NETWORKS,
                  input_demand_dir = INPUT_DEMAND,
                  run_config       = RUN_CONFIG,
                  input_weights    = INPUT_WEIGHTS,
                  output_dir       = OUTPUT_DIR,
                  output_folder    = OUTPUT_FOLDER,
                  pathfinding_type = PATHFINDING_TYPE,
                  iters            = ITERATIONS,
                  overlap_variable = OVERLAP,
                  dispersion       = DISPERSION,
                  capacity         = CAPACITY)

Uhoh - it looks like some of your passengers didn't make it!  Let's figure out what is going on.

# Examine Results

## Vehicle/Route Level
First, let's examine what is going on with the vehicles. 

In [ ]:
pd.set_option('display.max_colwidth',160) 
full_output_directory=os.path.join(OUTPUT_DIR,OUTPUT_FOLDER)
vehicles_df = pd.read_csv(os.path.join(full_output_directory,"veh_trips.csv"), 
                                       sep=",", 
                                       parse_dates=['arrival_time', 'departure_time'],
                                       date_parser=lambda x: datetime.datetime.strptime(x, '%H:%M:%S') )

vehicles_df[['iteration','pathfinding_iteration','arrival_time','departure_time','route_id','trip_id','stop_id','boards','onboard','standees']]

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

sns.set(style="whitegrid")
fig, axs = plt.subplots(nrows=3,figsize=(15,15))

iter1 = sns.barplot(ax=axs[0], x="trip_id", order=["B1","B2","B3","G1","R1","G2","R2","G3","G4"],y="boards", hue="pathfinding_iteration",  estimator=sum, ci=None, data=vehicles_df[vehicles_df["iteration"]==1])
iter2 = sns.barplot(ax=axs[1], x="trip_id", order=["B1","B2","B3","G1","R1","G2","R2","G3","G4"],y="boards", hue="pathfinding_iteration",  estimator=sum, ci=None, data=vehicles_df[vehicles_df["iteration"]==2])
iter3 = sns.barplot(ax=axs[2], x="trip_id", order=["B1","B2","B3","G1","R1","G2","R2","G3","G4"],y="boards", hue="pathfinding_iteration",  estimator=sum, ci=None, data=vehicles_df[vehicles_df["iteration"]==3])

axs[0].set_title('Iteration #1', fontsize=24,color="Gray")
axs[1].set_title('Iteration #2', fontsize=24,color="Gray")
axs[2].set_title('Iteration #3', fontsize=24,color="Gray")

There are two types of iterations going on here: pathfinding and global.

### Pathfinding Iterations
Pathfinding iterations allow passengers who were denied boarding a chance to go back and see if any of the other paths in their choice-set have availability.  You can see that two passengers who probably wanted to initially board B2 decided to get on B3 and G4 after learning that B2 was full in the initial pathfinding iteration.

### Global Iterations
Global iterations go through a travel-time-updating step and then re-initiate pathfinding.  In this example, all the possible paths are found in the first iteration and there is not much additional wiggle room for a passenger to re-optimize.

However, it looks like there is a tradeoff between G3 and R2 for the ten individuals who fit on the B2 bus.  They are  trading off in-vehicle and out of vehicle time.

### Who missed the bus?

It is evident that both B2 and B3 are at capacity, and so some of our friends didn't make it. Let's find out what that looks like in the path results.

## Person Level

In [ ]:
PATHS  = r"pathset_paths.csv"

pathset_paths_df = pd.read_csv(os.path.join(full_output_directory,PATHS), sep=",")
pd.set_option('display.max_colwidth',160) #widen so you can see whole description

In [ ]:
mswuf = pathset_paths_df[pathset_paths_df['person_id']=='Mrs. Wuf']
mswuf[['person_id','iteration','pathfinding_iteration','pathnum','probability','chosen','description']]

As you can see, Mrs. Wuf originally tried to board the B2 but was rejected becuase it was full.  

The next **pathfinding** iteration, she was able to board the B3 instead.

However, in next large iteration, she re-tries the B2 [ why not? it is still a better path for her if she gets there before Shammond Williams] and is then redirected again to the B3.  

In [ ]:
carlos = pathset_paths_df[pathset_paths_df['person_id']=='Carlos Boozer']
carlos[['person_id','iteration','pathfinding_iteration','pathnum','probability','chosen','description']]

Carlos has a different issue: he only has one valid path: B3-->G4.
When people like Mrs. Wuf get put on B3, they have priority (they got there first) over Carlos, so he gets booted.

In [ ]:
rejected_persons_df = pathset_paths_df[pathset_paths_df['chosen']=='rejected']
rejected_persons_df = rejected_persons_df[['person_id']]
rejected_persons_paths_df = pd.merge(pathset_paths_df, rejected_persons_df, on=['person_id'], how='inner')
rejected_persons_paths_df[['person_id','iteration','pathfinding_iteration','pathnum','probability','chosen','description']]

## QUESTION

**Question 3-A** Who didn't make the trip by the end of iteration 3?


# Add a Route and Evaluate Cost of Crowding
In this scenario, we will add a route that goes between the origin and destination without a transfer.  However, there is a longer wait up front and a longer walk for the egress.  The later departure time should work well for the Duke Blue Devils who don't leave the basketball court until a few minutes after the Tarheels and Wolfpack.

In [ ]:
INPUT_NETWORKS_NEW_ROUTE   = os.path.join(BASE_DIR, r"tta/input/network-added-route")
OUTPUT_FOLDER_NEW_ROUTE    = r"capacity-demand-wNewRoute"

Run.run_fasttrips(input_network_dir    = INPUT_NETWORKS_NEW_ROUTE,
                  input_demand_dir = INPUT_DEMAND,
                  run_config       = RUN_CONFIG,
                  input_weights    = INPUT_WEIGHTS,
                  output_dir       = OUTPUT_DIR,
                  output_folder    = OUTPUT_FOLDER_NEW_ROUTE,
                  pathfinding_type = PATHFINDING_TYPE,
                  iters            = ITERATIONS,
                  overlap_variable = OVERLAP,
                  dispersion       = DISPERSION,
                  capacity         = CAPACITY)



Phew, it looks like everybody made it this time!  Let's take a look at how the ridership turned out.

In [ ]:
pd.set_option('display.max_colwidth',160) 
full_output_directory=os.path.join(OUTPUT_DIR,OUTPUT_FOLDER_NEW_ROUTE)
vehicles_df = pd.read_csv(os.path.join(full_output_directory,"veh_trips.csv"), 
                                       sep=",", 
                                       parse_dates=['arrival_time', 'departure_time'],
                                       date_parser=lambda x: datetime.datetime.strptime(x, '%H:%M:%S') )

vehicles_df[['iteration','pathfinding_iteration','arrival_time','departure_time','route_id','trip_id','stop_id','boards','onboard','standees']]

import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

sns.set(style="whitegrid")
fig, axs = plt.subplots(nrows=3,figsize=(15,15))

iter1 = sns.barplot(ax=axs[0], x="trip_id", order=["O1","B1","B2","B3","G1","R1","G2","R2","G3","G4"],y="boards", hue="pathfinding_iteration",  estimator=sum, ci=None, data=vehicles_df[vehicles_df["iteration"]==1])
iter2 = sns.barplot(ax=axs[1], x="trip_id", order=["O1","B1","B2","B3","G1","R1","G2","R2","G3","G4"],y="boards", hue="pathfinding_iteration",  estimator=sum, ci=None, data=vehicles_df[vehicles_df["iteration"]==2])
iter3 = sns.barplot(ax=axs[2], x="trip_id", order=["O1","B1","B2","B3","G1","R1","G2","R2","G3","G4"],y="boards", hue="pathfinding_iteration",  estimator=sum, ci=None, data=vehicles_df[vehicles_df["iteration"]==3])

axs[0].set_title('Iteration #1', fontsize=24,color="Gray")
axs[1].set_title('Iteration #2', fontsize=24,color="Gray")
axs[2].set_title('Iteration #3', fontsize=24,color="Gray")

Well it looks as though the new "Orange Express" is a winner!  It is the clear favorite in each of the first path-finding iterations and then ridership spreads out to accommodate second and third priorities.

One of the interesting analyses that you can do in this regard is to evaluate the effect of the disutility of having to take a "next-best" path rather than your optimal path due to crowding.

## QUESTIONS

**Question 3-B** How would you measure the cost of crowding in the scenarios both before and after adding the "orange express"?

**Question 3-C** What is the cost of crowding with the orange line?